In [193]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline 

In [194]:
train=pd.read_csv('./crx_data_train_x.csv', header=None) #, na_values='?')
train_y=pd.read_csv('./crx_data_train_y.csv', header=None)

In [195]:
train['target']=train_y[1]


In [196]:
train.info()
# полезно для pd.read_csv('./crx_data_train_x.csv', header=None, na_values='?')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 483 entries, 0 to 482
Data columns (total 17 columns):
0         483 non-null int64
1         483 non-null object
2         483 non-null object
3         483 non-null float64
4         483 non-null object
5         483 non-null object
6         483 non-null object
7         483 non-null object
8         483 non-null float64
9         483 non-null object
10        483 non-null object
11        483 non-null int64
12        483 non-null object
13        483 non-null object
14        483 non-null object
15        483 non-null int64
target    483 non-null object
dtypes: float64(2), int64(3), object(12)
memory usage: 67.9+ KB


In [197]:
train[train['target']=='-'].groupby(1).count()

0    2    3    4    5    6    7    8    9   10   11   12   13   14   15  \
1                                                                              
?    6    6    6    6    6    6    6    6    6    6    6    6    6    6    6   
a   75   75   75   75   75   75   75   75   75   75   75   75   75   75   75   
b  180  180  180  180  180  180  180  180  180  180  180  180  180  180  180   

   target  
1          
?       6  
a      75  
b     180

for i in range(0,16):    
    first = train.groupby(i)[i].count()
    first.sort(ascending=False)
    fig = plt.figure(figsize=(10,2))
    ax1 = fig.add_subplot(121)
    first.plot(kind='bar')

In [198]:
contin=[0,2,3,8,11,14,15]
categ=[1,4,5,6,7,9,10,12,13]
name=categ+contin

### забацаем заполнение пропущ в колич фичах - средними, а в категориальных так и оставим отдельным классом

In [199]:
dt=train.copy()
name=[i for i in range(0,16)]
for i in contin[:]:
    if  type(dt[name[i]][1])==str:
        dt[name[i]]=pd.Series(dt[dt[name[i]]!='?'][name[i]].astype(float)) 
mean_for_missing_values= dt.mean()
print dt.mean()
dt=dt.fillna(dt.mean())


0     341.939959
2      31.555462
3       4.796056
8       2.171170
11      2.387164
14    184.981132
15    951.000000
dtype: float64


In [200]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 483 entries, 0 to 482
Data columns (total 17 columns):
0         483 non-null int64
1         483 non-null object
2         483 non-null float64
3         483 non-null float64
4         483 non-null object
5         483 non-null object
6         483 non-null object
7         483 non-null object
8         483 non-null float64
9         483 non-null object
10        483 non-null object
11        483 non-null int64
12        483 non-null object
13        483 non-null object
14        483 non-null float64
15        483 non-null int64
target    483 non-null object
dtypes: float64(4), int64(3), object(10)
memory usage: 67.9+ KB


In [201]:
c=dt.groupby(0)[0].count()
c[c>1]
# id шники уникальны

Series([], Name: 0, dtype: int64)

import seaborn as sns
dt=dt.dropna()
sns.pairplot(dt,hue='target')

# ONE-HOT ENCODING FOR categorical values

In [202]:
# one-hot encoding
dt['id']=pd.Series(dt.index)
for ii in categ:
    i=name[ii]
    n=dt[i]
    k=pd.get_dummies(n, prefix=i)
    k.head()
    k['id']=pd.Series(k.index)
    dt=pd.merge(dt,k)

    n=list(dt.columns.values)
    e=n.index(i)
    dt=dt.loc[:,n[:e]+n[e+1:]]
#отредактируем еще таргет
n=dt.target
k=pd.get_dummies(n, prefix='target')

k['id']=pd.Series(k.index)
k=k.drop('target_-', axis=1)
dt=pd.merge(dt,k)

n=list(dt.columns.values)
e=n.index('target')
dt=dt.loc[:,n[:e]+n[e+1:]]
dt.columns.values[-1]='target'
dt=dt.drop(0, axis=1)
dt=dt.drop('id', axis=1)

In [203]:
dt

2       3       8  11   14    15  1_?  1_a  1_b  4_?   ...    \
0    35.580000   0.750   1.500   0  231     0    0    0    1    0   ...     
1    34.920000   2.500   0.000   0  239   200    0    0    1    0   ...     
2    31.555462   4.000   0.085   0  411     0    0    0    1    0   ...     
3    46.080000   3.000   2.375   8  396  4159    0    1    0    0   ...     
4    23.580000   0.830   0.415   1  200    11    0    1    0    0   ...     
5    23.000000   0.750   0.500   0  320     0    0    0    1    0   ...     
6    23.250000   1.500   2.375   3    0   582    0    0    1    0   ...     
7    22.920000   1.250   0.250   0  120   809    0    1    0    0   ...     
8    38.670000   0.210   0.085   0  280     0    0    0    1    0   ...     
9    28.670000   1.040   2.500   5  300  1430    0    1    0    0   ...     
10   26.170000  12.500   1.250   0    0    17    0    0    1    0   ...     
11   31.555462   0.375   0.875   0  928     0    0    0    1    0   ...     
12   18.920000   9.250   1.000   4   80   500    0    1    0    0   ...     
13   21.580000   0.790   0.665   0  160     0    0    0    1    0   ...     
14   80.250000   5.500   0.540   0    0   340    0    0    1    0   ...     
15   20.830000   3.000   0.040   0  100     0    0    1    0    0   ...     
16   56.420000  28.000  28.500  40    0    15    0    0    1    0   ...     
17   22.670000  10.500   1.335   0  100     0    0    0    1    0   ...     
18   22.670000   2.540   2.585   0    0     0    0    0    1    0   ...     
19   22.330000  11.000   2.000   1   80   278    0    0    1    0   ...     
20   51.330000  10.000   0.000  11    0  1249    0    0    1    0   ...     
21   21.170000   0.875   0.250   0  280   204    0    0    1    0   ...     
22   26.920000  13.500   5.000   2    0  5000    0    1    0    0   ...     
23   25.080000   1.710   1.665   1  395    20    0    0    1    0   ...     
24   27.830000   1.000   3.000   0  176   537    0    0    1    0   ...     
25   23.080000   0.000   1.000  11    0     0    0    0    1    0   ...     
26   34.750000   2.500   0.500   0  348     0    0    0    1    0   ...     
27   30.750000   1.585   0.585   0    0     0    0    0    1    0   ...     
28   41.580000   1.750   0.210   0  160     0    0    0    1    0   ...     
29   23.920000   1.500   1.875   6  200   327    0    0    1    0   ...     
..         ...     ...     ...  ..  ...   ...  ...  ...  ...  ...   ...     
453  31.555462   0.040   4.250   0  460     0    0    0    1    0   ...     
454  21.500000   9.750   0.250   0  140     0    0    0    1    0   ...     
455  34.920000   5.000   7.500   6    0  1000    0    0    1    0   ...     
456  28.330000   5.000  11.000   0   70     0    0    0    1    0   ...     
457  33.000000   2.500   7.000   0  280     0    0    0    1    0   ...     
458  30.170000   1.085   0.040   0  170   179    0    0    1    0   ...     
459  39.080000   6.000   1.290   5  108  1097    0    0    1    0   ...     
460  38.920000   1.665   0.250   0    0   390    0    1    0    0   ...     
461  28.750000   3.750   1.085   1  371     0    0    1    0    0   ...     
462  16.000000   3.125   0.085   1    0     6    0    0    1    0   ...     
463  17.920000   0.205   0.040   0  280   750    0    0    1    0   ...     
464  27.670000   1.500   2.000   0  368     0    0    1    0    0   ...     
465  22.080000   2.335   0.750   0  180     0    0    0    1    0   ...     
466  47.420000   3.000  13.875   2  519  1704    0    1    0    0   ...     
467  39.000000   5.000   3.500  10    0     0    0    0    1    0   ...     
468  23.000000  11.750   0.500   2  300   551    0    1    0    0   ...     
469  31.250000   3.750   0.625   9  181     0    0    1    0    0   ...     
470  23.580000   0.460   2.625   6  208   347    0    0    1    0   ...     
471  19.500000   9.585   0.790   0   80   350    0    0    1    0   ...     
472  26.170000   0.250   0.000   0    0     0    0    0    1    0   ...     
473  22.5

In [13]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline 
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin
from sklearn import preprocessing

In [20]:
train=pd.read_csv('./crx_data_train_x.csv', header=None) #, na_values='?')
train_y=pd.read_csv('./crx_data_train_y.csv', header=None)
train['target']=train_y[1]

contin=[0,2,3,8,11,14,15]
categ=[1,4,5,6,7,9,10,12,13]
name=categ+contin

dt=train.copy()

In [21]:


class My_transformer(TransformerMixin):

    def transform(self, dt, **transform_params):

        contin=[0,2,3,8,11,14,15]
        categ=[1,4,5,6,7,9,10,12,13]
        name=categ+contin
        name=[i for i in range(0,16)]
        for i in contin[:]:
            if  type(dt[name[i]][1])==str:
                dt[name[i]]=pd.Series(dt[dt[name[i]]!='?'][name[i]].astype(float)) 
        mean_for_missing_values= dt.mean()
        #print dt.mean()

        dt = dt.fillna(dt.mean())

        contin=[0,2,3,8,11,14,15]
        categ=[1,4,5,6,7,9,10,12,13]
        name=categ+contin
        dt['id']=pd.Series(dt.index)
        for ii in categ:
            i=name[ii]
            n=dt[i]
            k=pd.get_dummies(n, prefix=i)
            k.head()
            k['id']=pd.Series(k.index)
            dt=pd.merge(dt,k)

            n=list(dt.columns.values)
            e=n.index(i)
            dt=dt.loc[:,n[:e]+n[e+1:]]
        #отредактируем еще таргет
        n=dt.target
        k=pd.get_dummies(n, prefix='target')

        k['id']=pd.Series(k.index)
        k=k.drop('target_-', axis=1)
        dt=pd.merge(dt,k)

        n=list(dt.columns.values)
        e=n.index('target')
        dt=dt.loc[:,n[:e]+n[e+1:]]
        dt.columns.values[-1]='target'
        dt=dt.drop(0, axis=1)
        dt=dt.drop('id', axis=1)

        y=np.hstack(np.array([dt['target'].values.astype(int)]))

        n=list(dt.columns.values)

        n=n[:n.index('target')]+n[n.index('target')+1:]

        x= np.vstack([np.array(dt[n[0]].values)])
        x=np.append(x, np.vstack([np.array(dt[n[42]].values)]),axis=0)
        x=np.append(x, np.vstack([np.array(dt[n[43]].values)]),axis=0)

        x=np.append(x, np.vstack([np.array(dt[n[42]].values)]),axis=0)
        x=np.append(x, np.vstack([np.array(dt[n[43]].values)]),axis=0)

        x=np.append(x, np.vstack([np.array(dt[n[3]].values)]),axis=0)
        x=np.append(x, np.vstack([np.array(dt[n[44]].values)]),axis=0)

        for i in n[:-5]:
            x=np.append(x, np.vstack([np.array(dt[i].values)]),axis=0)
        x=x.T

        x_scaled = preprocessing.scale(x)
        return x,y
    
    def fit(self, X, y=None, **fit_params):
        return self
    


In [18]:
pipl = Pipeline([
        
        ('my_features', My_transformer()),
        ('svm', svm.SVC())
    ])

In [22]:
pipl.fit(dt)

0     341.939959
2      31.555462
3       4.796056
8       2.171170
11      2.387164
14    184.981132
15    951.000000
dtype: float64


ValueError: labels [0] not contained in axis

### создадим себе выборки хорошие для тренировок

In [204]:
y=np.hstack(np.array([dt['target'].values.astype(int)]))
print y.shape

n=list(dt.columns.values)
#qq=set(n)
n=n[:n.index('target')]+n[n.index('target')+1:]
#print qq-set(n)
 
x= np.vstack([np.array(dt[n[0]].values)])
x=np.append(x, np.vstack([np.array(dt[n[42]].values)]),axis=0)
x=np.append(x, np.vstack([np.array(dt[n[43]].values)]),axis=0)

x=np.append(x, np.vstack([np.array(dt[n[42]].values)]),axis=0)
x=np.append(x, np.vstack([np.array(dt[n[43]].values)]),axis=0)

x=np.append(x, np.vstack([np.array(dt[n[3]].values)]),axis=0)
x=np.append(x, np.vstack([np.array(dt[n[44]].values)]),axis=0)




for i in n[:-5]:
    x=np.append(x, np.vstack([np.array(dt[i].values)]),axis=0)
print x.shape
x=x.T

(483,)
(52, 483)


### scale

In [205]:
from sklearn import preprocessing
x_scaled = preprocessing.scale(x)

In [206]:

from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=2,n_estimators=50, random_state=1, min_samples_leaf=3,max_depth=10)
rf.fit(x_scaled,y)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=3, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=2,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

In [142]:
a=rf.feature_importances_

In [143]:
re=np.argsort(a)
for i in re[::-1]:
    print i,' ', a[i]

43   0.221414038728
42   0.189158120155
3   0.0764698960614
44   0.0655317143907
4   0.0628299152497
6   0.0550094463012
2   0.0468537834739
5   0.0434278732012
45   0.0418754111993
1   0.0331937455905
0   0.0331290089579
37   0.0112928944756
32   0.00836264457311
40   0.00682198570697
24   0.0067135271891
29   0.00656866516252
36   0.00621483769325
46   0.00604575242343
47   0.00582003093314
17   0.00552837358667
12   0.00552393225958
50   0.00533849450598
27   0.00468836370871
9   0.00464972428769
48   0.00435062067896
8   0.00432138459365
14   0.0042507774772
15   0.00407023392293
13   0.00375499229862
20   0.00363272347325
25   0.0034011503877
21   0.00339437933501
34   0.00330097170012
31   0.00313925359451
19   0.00228120039017
10   0.00173911362584
33   0.00158913437639
18   0.0012281699713
49   0.00106896504279
28   0.000863185596478
22   0.000525170417935
23   0.000425613051392
26   0.000167694037595
35   3.30862137335e-05
11   0.0
7   0.0
16   0.0
38   0.0
39   0.0
41   0.0
3

In [77]:
print len(a)
print max(a)

print a[42]
print a[43]
print a[3]
print 
print map(lambda x: round(x,5),a)

54
0.190467300108
0.00341552019195
0.00844587158959
0.0494428847034

[0.03984, 0.01407, 0.02339, 0.04944, 0.04119, 0.04917, 0.0703, 0.05531, 0.049, 0.045, 0.00022, 0.00731, 0.00592, 0.00288, 0.00145, 0.00723, 0.0062, 0.00273, 0.00637, 0.00062, 0.00531, 0.00153, 0.00362, 0.00684, 0.00302, 0.00186, 0.00155, 0.00594, 0.004, 0.0005, 0.00903, 0.00263, 0.00733, 0.00027, 0.00615, 0.00805, 0.00246, 0.00636, 0.00011, 0.0033, 0.00842, 0.00199, 0.00342, 0.00845, 0.00079, 0.19047, 0.15153, 0.01553, 0.034, 0.00734, 0.00642, 0.00476, 0.0022, 0.00718]


In [84]:
a=cross_val_score(RandomForestClassifier(n_jobs=2,n_estimators=100, random_state=4, min_samples_leaf=1), x_scaled, y, cv=3, scoring='accuracy')
print np.mean(a)

0.865424430642


In [17]:
from sklearn import grid_search
from sklearn.cross_validation import cross_val_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm

## Random Forest

In [85]:
params_rf = {'n_estimators':[50,100,300], 'random_state':[1,2,3,4,5], 'min_samples_leaf':[1,2,3,4], 'max_depth':[None, 10,30,50]}
rf = RandomForestClassifier(n_jobs=2)

In [86]:
rf_gs=grid_search.GridSearchCV(rf, params_rf, scoring='accuracy')

In [87]:
rf_gs.fit(x_scaled,y)

GridSearchCV(cv=None, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'n_estimators': [50, 100, 300], 'random_state': [1, 2, 3, 4, 5], 'max_depth': [None, 10, 30, 50], 'min_samples_leaf': [1, 2, 3, 4]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring='accuracy', verbose=0)

In [88]:
print rf_gs.best_score_
print rf_gs.best_params_

0.871635610766
{'n_estimators': 50, 'random_state': 1, 'max_depth': 10, 'min_samples_leaf': 3}


In [108]:
a=cross_val_score(RandomForestClassifier(n_estimators=50, random_state=1, min_samples_leaf=3), x_scaled, y, cv=2, scoring='accuracy')
print np.mean(a)

0.861287335825


In [171]:
rf = RandomForestClassifier(n_jobs=2, n_estimators=50, random_state=1, min_samples_leaf=3, max_depth=10)
rf.fit(x_scaled,y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=3, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=2,
            oob_score=False, random_state=1, verbose=0, warm_start=False)

## svm

In [183]:
param_svm = dict(gamma=[0.00001, 0.00005, 0.0001,0.001, 0.1,1,10], C=[1,5,7,10,15] )
svr = svm.SVC()
grid = grid_search.GridSearchCV(svr, param_svm,scoring='accuracy')

In [184]:
grid.fit(x_scaled,y)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'C': [1, 5, 7, 10, 15], 'gamma': [1e-05, 5e-05, 0.0001, 0.001, 0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring='accuracy', verbose=0)

In [185]:
print grid.best_score_
print grid.best_params_

0.867494824017
{'C': 7, 'gamma': 5e-05}


In [187]:
print np.mean(cross_val_score(svm.SVC(C = 5, gamma = 0.0001, kernel='rbf'), x_scaled, y, cv=4,scoring='accuracy'))

0.852990787156


### knn

In [190]:
params_knn={'n_neighbors':[1,10,20,42,43,44,50]}
knn=KNeighborsClassifier()
gs3=grid_search.GridSearchCV(knn, params_knn, scoring='accuracy')


In [191]:
gs3.fit(x_scaled,y)

GridSearchCV(cv=None, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_neighbors=5, p=2, weights='uniform'),
       fit_params={}, iid=True, loss_func=None, n_jobs=1,
       param_grid={'n_neighbors': [1, 10, 20, 42, 43, 44, 50]},
       pre_dispatch='2*n_jobs', refit=True, score_func=None,
       scoring='accuracy', verbose=0)

In [192]:
print gs3.best_score_
print gs3.best_params_

0.857142857143
{'n_neighbors': 50}


### считаем тест, объединим его с трейном, глянем чтобы при one-hot у нас фичей столько же оказалось. 

>>> стоило бы заюзать какую нибудь функцию feature encoding чтобы не возиться так

In [207]:
test=pd.read_csv('./crx_data_test_x.csv', header=None)

In [208]:
dt=test.copy()
name=[i for i in range(0,16)]
for i in contin[:]:
    if  type(dt[name[i]][1])==str:
        dt[name[i]]=pd.Series(dt[dt[name[i]]!='?'][name[i]].astype(float)) 
dt=dt.fillna(mean_for_missing_values)



In [209]:
dt_fr=[train, dt]
res=pd.concat(dt_fr,ignore_index=True)
dt=res
res[res.index==482]

0  1      2   3  4  5   6   7  8  9 10  11 12 13    14  15 target
482  80  b  49.58  19  u  g  ff  ff  0  t  t   1  f  g  94.0   0      -

In [210]:
# one-hot encoding
dt['id']=pd.Series(dt.index)
for ii in categ:
    i=name[ii]
    n=dt[i]
    k=pd.get_dummies(n, prefix=i)
    k.head()
    k['id']=pd.Series(k.index)
    dt=pd.merge(dt,k)

    n=list(dt.columns.values)
    e=n.index(i)
    dt=dt.loc[:,n[:e]+n[e+1:]]

#отредактируем еще таргет

n=list(dt.columns.values)
e=n.index('target')
dt=dt.loc[:,n[:e]+n[e+1:]]
dt=dt.drop(0, axis=1)
dt=dt.drop('id', axis=1)
dt=dt.drop('7_o', axis=1)
dt.head(10)

2     3      8  11     14    15  1_?  1_a  1_b  4_?  ...   7_z  9_f  \
0  35.58  0.75  1.500   0  231.0     0    0    0    1    0  ...     0    1   
1  34.92  2.50  0.000   0  239.0   200    0    0    1    0  ...     0    0   
2      ?  4.00  0.085   0  411.0     0    0    0    1    0  ...     0    1   
3  46.08  3.00  2.375   8  396.0  4159    0    1    0    0  ...     0    0   
4  23.58  0.83  0.415   1  200.0    11    0    1    0    0  ...     0    1   
5   23.0  0.75  0.500   0  320.0     0    0    0    1    0  ...     0    0   
6  23.25  1.50  2.375   3    0.0   582    0    0    1    0  ...     0    0   
7  22.92  1.25  0.250   0  120.0   809    0    1    0    0  ...     0    1   
8  38.67  0.21  0.085   0  280.0     0    0    0    1    0  ...     0    0   
9  28.67  1.04  2.500   5  300.0  1430    0    1    0    0  ...     0    0   

   9_t  10_f  10_t  12_f  12_t  13_g  13_p  13_s  
0    0     1     0     0     1     1     0     0  
1    1     1     0     0     1     1     0     0  
2    0     1     0     0     1     1     0     0  
3    1     0     1     0     1     1     0     0  
4    0     0     1     0     1     1     0     0  
5    1     1     0     0     1     0     0     1  
6    1     0     1     0     1     1     0     0  
7    0     1     0     0     1     1     0     0  
8    1     1     0     0     1     1     0     0  
9    1     0     1     0     1     1     0     0  

[10 rows x 50 columns]

In [211]:
dt=dt[dt.index>482]

In [214]:
n=list(dt.columns.values)
print n
#print n.index('7_o')
#n=n[:n.index('id')]+n[n.index('id')+1:]
#n=n[:n.index('7_o')]+n[n.index('7_o')+1:]

print n[43]
x= np.vstack([np.array(dt[n[0]].values)])
x=np.append(x, np.vstack([np.array(dt[n[42]].values)]),axis=0)
x=np.append(x, np.vstack([np.array(dt[n[43]].values)]),axis=0)

x=np.append(x, np.vstack([np.array(dt[n[42]].values)]),axis=0)
x=np.append(x, np.vstack([np.array(dt[n[43]].values)]),axis=0)

x=np.append(x, np.vstack([np.array(dt[n[3]].values)]),axis=0)
x=np.append(x, np.vstack([np.array(dt[n[44]].values)]),axis=0)



for i in n[:-5]:
    x=np.append(x, np.vstack([np.array(dt[i].values)]),axis=0)
print x.shape
x=x.T

[2, 3, 8, 11, 14, 15, '1_?', '1_a', '1_b', '4_?', '4_l', '4_u', '4_y', '5_?', '5_g', '5_gg', '5_p', '6_?', '6_aa', '6_c', '6_cc', '6_d', '6_e', '6_ff', '6_i', '6_j', '6_k', '6_m', '6_q', '6_r', '6_w', '6_x', '7_?', '7_bb', '7_dd', '7_ff', '7_h', '7_j', '7_n', '7_v', '7_z', '9_f', '9_t', '10_f', '10_t', '12_f', '12_t', '13_g', '13_p', '13_s']
10_f
(52, 207)


In [215]:
set([0, 2, 3, 8, 11, 14, 15, 'id', '1_?', '1_a', '1_b', '4_?', '4_l', '4_u', '4_y', '5_?', '5_g', '5_gg', '5_p', '6_?', '6_aa', '6_c', '6_cc', '6_d', '6_e', '6_ff', '6_i', '6_j', '6_k', '6_m', '6_q', '6_r', '6_w', '6_x', '7_?', '7_bb', '7_dd', '7_ff', '7_h',
 '7_j', '7_n', '7_v', '7_z', '9_f', '9_t', '10_f', '10_t', '12_f', '12_t', '13_g', '13_p', '13_s', 'target']) - set([0, 2, 3, 8, 11, 14, 15, 'id', '1_?', '1_a', '1_b', '4_?', '4_l', '4_u', '4_y', '5_?', '5_g', '5_gg', '5_p', '6_?', '6_aa', '6_c', '6_cc', '6_d', '6_e', '6_ff', '6_i', '6_j', '6_k', '6_m', '6_q', '6_r', '6_w', '6_x', '7_?', '7_bb', '7_dd', '7_ff', '7_h', '7_j', '7_n', '7_o', '7_v', '7_z', '9_f', '9_t', '10_f', '10_t', '12_f', '12_t', '13_g', '13_p', '13_s'])

{'target'}

In [216]:
set([2, 3, 8, 11, 14, 15, 'id', '1_?', '1_a', '1_b', '4_?', '4_l', '4_u', '4_y', '5_?', '5_g', '5_gg', '5_p', '6_?', '6_aa', '6_c', '6_cc', '6_d', '6_e', '6_ff', '6_i', '6_j', '6_k', '6_m', '6_q', '6_r', '6_w', '6_x', '7_?', '7_bb', '7_dd', '7_ff', '7_h', '7_j', '7_n', '7_o', '7_v', '7_z', '9_f', '9_t', '10_f', '10_t', '12_f', '12_t', '13_g', '13_p', '13_s'])-set([2, 3, 8, 11, 14, 15, '1_?', '1_a', '1_b', '4_?', '4_l', '4_u', '4_y', '5_?', '5_g', '5_gg', '5_p', '6_?', '6_aa', '6_c', '6_cc', '6_d', '6_e', '6_ff', '6_i', '6_j', '6_k', '6_m', '6_q', '6_r', '6_w', '6_x', '7_?', '7_bb', '7_dd', '7_ff', '7_h', '7_j', '7_n', '7_v', '7_z', '9_f', '9_t', '10_f', '10_t', '12_f', '12_t', '13_g', '13_p', '13_s'])

{'7_o', 'id'}

In [217]:
x = preprocessing.scale(x)

## какая то путаница с ответом, так (если поменять наоборот ответы придиктора то ), результат около 0,752 будет. иначе 0,248

In [218]:
ans = pd.Series(rf.predict(x))
res=pd.DataFrame()
ans[ans==1]='0'
ans[ans==0]='1'

In [219]:
#res['id']=test[0]
res['y']=ans
res
res.to_csv('ans.txt', index=False, header=None )

In [ ]:
## чисто 0657 получается, но я по случайности добавил первую фичу два раза - - - в итоге прирост точности хм)

In [ ]:
## дали первую фичу дважды - 0,752 

In [ ]:
## дали еще фичу 10_f прирост до 0.781 

In [61]:
## сделал дважды 42 и еще 43 и 3 фичи, чтобы вероятость попадания в дерево была выше.
## их нашел по фичер импортансу из модели rf

# 0,714

> 0,867 выдало это решение. задача тестовая, упираться не буду)

In [42]:
## 43 44 4 - 0,867